<a href="https://colab.research.google.com/github/CarolineMNves/MVP_PUC/blob/main/Projeto_de_Machine_Learning_HTRU2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introdução

  Este documento tem como objetivo contemplar a avaliação da disciplina de Machine Learning, do curso de Pós-Graduação em Ciência de Dados e Analytics da instituição Pontifícia Universidade Católica do Rio de Janeiro.
O projeto tem como objetivo construir um modelo de Machine Learning, utilizando o conjunto de dados HTRU2, para identificar se um objeto celeste trata-se de um Pulsar.

# 2. O conjunto de dados
O HTRU2 é um conjunto de dados que descreve uma amostra de candidatos a pulsar, coletados durante a 	High Time Resolution Universe Survey.
Pulsares são estrelas de nêutrons que produzem emissões de rádio detectáveis aqui na Terra. Seus padrões são detectáveis devido ao seu movimento - ao girarem, seu feixe de emissão varre o céu, produzindo um padrão detectável de emissão de rádio; a velocidade de seu movimento causa uma peridiciocidade neste padrão.
No conjunto, cada candidato é descrito por oito variáveis contínuas e, os exemplos legítimos de pulsares são uma classe positiva minoritária, enquanto que os exemplos hipotéticos à classe negativa são majoritários.

# 3. Análise Inicial
Para começar, vamos importar as bibliotecas Pandas, Numpy,Files do GoogleColab e MatplotLib, necessárias para o carregamento e análise inicial dos dados.
Utilizamos o método 'files.upload' para o download do arquivo no ambiente. Em seguida, criamos uma lista para armazenar os nomes de cada coluna do dataset e utilizamos o método 'pd.read_csv' para ler o dataset.
Com o dataset carregado, chamamos métodos como 'info.()' e 'describe.()' para obtermos descrições dos dados, a fim de entender algumas de suas dimensões.

In [1]:
#importando bibliotecas
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
#carregando o arquivo
uploaded = files.upload()

Saving HTRU_2.csv to HTRU_2.csv


In [3]:
#nomes das colunas

column_names = ['mean_profile', 'std_profile', 'kurtosis_profile', 'skewness_profile',
                'mean_dm', 'std_dm', 'kurtosis_dm', 'skewness_dm', 'target_class']

pulsar_data = pd.read_csv(next(iter(uploaded)), names=column_names)



In [4]:
#verificando o carregamento dos dados
pulsar_data.head()

,mean_profile,std_profile,kurtosis_profile,skewness_profile,mean_dm,std_dm,kurtosis_dm,skewness_dm,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [5]:
#Quantidade de dados em cada coluna e seus respectivos formatos
pulsar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mean_profile      17898 non-null  float64
 1   std_profile       17898 non-null  float64
 2   kurtosis_profile  17898 non-null  float64
 3   skewness_profile  17898 non-null  float64
 4   mean_dm           17898 non-null  float64
 5   std_dm            17898 non-null  float64
 6   kurtosis_dm       17898 non-null  float64
 7   skewness_dm       17898 non-null  float64
 8   target_class      17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


In [6]:
#Visualizando as estatísticas dos dados
pulsar_data.describe()

,mean_profile,std_profile,kurtosis_profile,skewness_profile,mean_dm,std_dm,kurtosis_dm,skewness_dm,target_class
count,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,111.079968,46.549532,0.477857,1.770279,12.614400,26.326515,8.303556,104.857709,0.091574
std,25.652935,6.843189,1.064040,6.167913,29.472897,19.470572,4.506092,106.514540,0.288432
min,5.812500,24.772042,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.929688,42.376018,0.027098,-0.188572,1.923077,14.437332,5.781506,34.960504,0.000000
50%,115.078125,46.947479,0.223240,0.198710,2.801839,18.461316,8.433515,83.064556,0.000000
75%,127.085938,51.023202,0.473325,0.927783,5.464256,28.428104,10.702959,139.309330,0.000000
max,192.617188,98.778911,8.069522,68.101622,223.392141,110.642211,34.539844,1191.000837,1.000000


#4. Visualizando os dados
aqui vou bostar uns gráficos *)

In [7]:
#importando bibliotecas necessária para constrção do modelo

from sklearn.model_selection import train_test_split # Holdout
from sklearn.model_selection import KFold # Folds da validação cruzada
from sklearn.model_selection import cross_val_score # Validação cruzada
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score # Acurácia do modelo
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix, auc, roc_curve #Avaliando o modelo
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report #Avaliando o modelo
from sklearn.ensemble import ExtraTreesClassifier # ExtraTrees, para a Importância de Atributos
from sklearn.feature_selection import SelectKBest # Para a Seleção Univariada
from sklearn.feature_selection import f_classif # Para o teste ANOVA da Seleção Univariada
from sklearn.feature_selection import RFE # Para a Eliminação Recursiva de Atributos
from sklearn.ensemble import BaggingClassifier #Evitando overfitting
from sklearn.ensemble import VotingClassifier #Ensemble
from sklearn.ensemble import AdaBoostClassifier #Adaptando os pesos dos exemplos de treinamento

from sklearn.neighbors import KNeighborsClassifier # Algoritmo KNN
from sklearn.tree import DecisionTreeClassifier # Algoritmo Árvore de Classificação
from sklearn.naive_bayes import GaussianNB # Algoritmo Naive Bayes
from sklearn.svm import SVC # Algoritmo SVM
from sklearn.linear_model import LogisticRegression # Algorito Regressão Logística
from sklearn.ensemble import GradientBoostingClassifier # Algoritmo Arvore de Decisão
from sklearn.ensemble import RandomForestClassifier # Algoritmo Random Forest




Separação em conjunto de treino e conjunto de teste com holdout

Neste momento a normalização dos dados já foi realizada. Ao testarmos o código inicialmente, não se fez possívl o carregamento do código, pois o algoritmo de rg logística é sensível à escala dos dados. Dessa forma, como os dados apresentam variações significativas de escala entre os atributos, aplicamos a técnica de normalização dos dados para tratar os dados e treinar o modelo

In [12]:
#Separação dos dados
array = pulsar_data.values
X = array[:, 0 : 8]
y = array[:, 8]

#Normalização dos dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

#Holdout com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size = 0.20, shuffle = True, random_state = 13, stratify=y)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(X_train), len(X_test)))

#Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits = num_particoes, shuffle = True, random_state = 13) # validação cruzada com estratificação

Treinaremos com 14318 elementos e testaremos com 3580 elementos


Criação e avaliação de modelos: linha base

In [13]:
#Definindo uma seed global
seed = 13

#Lista para armazenamento dos modelos
models = []

#Definindo os parâmetros do classificador base para o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 200
max_features = 5

#Criação e armazenamento dos modelos
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('LR', LogisticRegression()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('RF', RandomForestClassifier()))

#Criação dos modelos para o VotingClassifier
bases = []
model_1 = LogisticRegression(max_iter = 1000)
bases.append(('logistic', model_1))
model_2 = DecisionTreeClassifier()
bases.append(('cart', model_2))
model_3 = SVC()
bases.append(('svm', model_3))

#Criando os ensembles e adicionando-os na lista de modelos
models.append(('Bagging', BaggingClassifier(base_estimator = model_1, n_estimators = num_trees)))
models.append(('RF', RandomForestClassifier(n_estimators = num_trees, max_features = max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators = num_trees, max_features = max_features)))
models.append(('Ada', AdaBoostClassifier(n_estimators = num_trees)))
models.append(('GB', GradientBoostingClassifier(n_estimators = num_trees)))
models.append(('Voting', VotingClassifier(bases)))

# Listas para armazenar os resultados
results = []
names = []


In [ ]:
# Avaliação dos modelos
for name, model in models:
    cv_results_acc = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'accuracy')
    cv_results_prec = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'precision_macro')
    cv_results_rec = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'recall_macro')
    cv_results_f1 = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'f1_macro')
    cv_results_roc_auc = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'roc_auc')

    results.append(cv_results_acc)
    results.append(cv_results_prec)
    results.append(cv_results_rec)
    results.append(cv_results_f1)
    results.append(cv_results_roc_auc)

    names.append(name)

    #resultados
    msg = "%s\nAccuracy: %f (%f)\nPrecision: %f (%f)\nRecall: %f (%f)\nF1-score: %f (%f)\nROC AUC: %f (%f)\n" % (
        name, cv_results_acc.mean(), cv_results_acc.std(), cv_results_prec.mean(), cv_results_prec.std(),
        cv_results_rec.mean(), cv_results_rec.std(), cv_results_f1.mean(), cv_results_f1.std(),
        cv_results_roc_auc.mean(), cv_results_roc_auc.std())
    print(msg)


In [27]:
#importando bibliotecas
import plotly.graph_objects as go

# Métricas
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC AUC']

# Dados dos modelos
models_results = np.array(results)

# Definições do gráfico de radar para avaliação do modelo
fig = go.Figure()

for i, model_name in enumerate(names):
    model_data = models_results[i]
    fig.add_trace(go.Scatterpolar(r=model_data, theta=metrics, fill='toself', name=model_name))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True
)

fig.show()
